# Лабораторная работа #3
Выполнили:
* Меньшутин Алексей M3436
* Юрий Каменев M3436

In [14]:
!{sys.executable} -m pip install numpy

In [3]:
import numpy as np
from numpy import zeros as zeros

def coefficients_to_string(values: [float]):
    result = ""
    has_previos_not_zero = False
    for i, value in enumerate(values):
        if value == 0:
            continue

        if i != 0:
            if has_previos_not_zero:
                result += " "
                result += "-" if value < 0 else "+"
                result += " "
                if abs(value) != 1:
                    result += str(abs(value))
            else:
                if abs(value) != 1:
                    result += str(value)
                else:
                    if value < 0:
                        result += "-"
        else:
            if value == -1:
                result += "-"
            else:
                if value != 1:
                    result += str(value)

        result += f"x{i + 1}"
        has_previos_not_zero = True

    return result

class Function:
    coefficients: [float]
    optimize_max: bool

    def __init__(self, coefficients: [float], optimize_max: bool=True):
        self.coefficients = np.array(coefficients, dtype=float)
        self.optimize_max = optimize_max

    def negate(self):
        self.coefficients *= -1
        self.optimize_max = not self.optimize_max

    def __str__(self):
        result = coefficients_to_string(self.coefficients)
        result += " -> "
        result += "max" if self.optimize_max else "min"

        return result

class Constraint:
    coefficients: [float]
    value: float
    sign: int

    def __init__(self, coefficients: [float], value: float, sign: int=0):
        self.coefficients = np.array(coefficients, dtype=float)
        self.value = value
        self.sign = sign

    def sign_less(self):
        return self.sign < 0

    def sign_equals(self):
        return self.sign == 0

    def sign_greater(self):
        return self.sign > 0

    def set_sign_equals(self):
        self.sign = 0

    def sign_to_string(self):
        if self.sign_less():
            return "<="
        else:
            if self.sign == 0:
                return "="
            else:
                return ">="

    def negate(self):
        self.coefficients *= -1
        self.value *= -1
        self.sign *= -1

    def __str__(self):
        result = coefficients_to_string(self.coefficients)
        result += " " + self.sign_to_string()
        result += " " + str(self.value)

        return result

class LinearProgrammingTask:
    function: Function
    constraints: [Constraint]

    def __init__(self, function: Function, constraints: [Constraint]):
        self.function = function
        self.constraints = constraints

    def to_canonic_form(self):
        if not self.function.optimize_max:
            self.function.negate()

        free_variables_count = 0
        for constraint in self.constraints:
            if constraint.sign != 0:
                free_variables_count += 1
                if constraint.sign_greater():
                    constraint.negate()

        np.append(self.function.coefficients, zeros(free_variables_count))

        current_free_variables_count = 0
        for constraint in self.constraints:
            if constraint.sign == 0:
                np.append(constraint.coefficients, zeros(free_variables_count))
            else:
                np.append(constraint.coefficients, (zeros(current_free_variables_count)))
                np.append(constraint.coefficients, 1)
                np.append(constraint.coefficients, zeros(free_variables_count - current_free_variables_count - 1))
                current_free_variables_count += 1
                constraint.set_sign_equals()

            if constraint.value < 0:
                constraint.negate()

    def dimensions(self) -> (int, int):
        return len(self.function.coefficients), len(self.constraints)

    def __str__(self):
        result = str(self.function) + "\n"
        result += "\n".join(str(constraint) for constraint in self.constraints)

        return result

In [4]:
from math import inf

eps = 1e-10

def sign(i: float) -> float:
    if i >= 0.0:
        return 1
    return -1

def build_table(lpt: LinearProgrammingTask, basic_feasible_solution: [float], n: int, m: int) -> ([[float]], [int]):
    basis = extract_basis(lpt.constraints, basic_feasible_solution, n, m)
    table = fill_table(lpt.function, lpt.constraints, basis, n, m)

    return table, basis

def extract_basis(constraints: [Constraint], basic_feasible_solution: [float], n: int, m: int) -> [int]:
    basis = []
    for i, value in enumerate(basic_feasible_solution):
        if value != 0:
            basis.append(i)

    constraints.sort(reverse=True, key=lambda constraint: [constraint.coefficients[i] for i in basis])
    i = 0
    while True:
        if i >= m:
            break

        zero_coeffs_count = 0
        for value in constraints[i].coefficients:
            if value < eps:
                zero_coeffs_count += 1

        if zero_coeffs_count == len(constraints[i].coefficients):
            constraints.remove(constraints[i])
            i -= 1
            m -= 1
            continue

        basis_value = constraints[i].coefficients[basis[i]]
        constraints[i].coefficients /= basis_value
        constraints[i].value /= basis_value

        for j in range(m):
            if i == j:
                continue

            delta = constraints[j].coefficients[basis[i]]
            for k in range(len(constraints[j].coefficients)):
                constraints[j].coefficients[k] -= delta * constraints[i].coefficients[k]
            constraints[j].value -= delta * constraints[i].value

        constraints.sort(reverse=True, key=lambda c: [c.coefficients[i] for i in basis])
        i += 1

    return basis

def fill_table(function: Function, constraints: [Constraint], basis: [float], n, m) -> [[float]]:
    table = list(map(lambda constraint: [constraint.value] + constraint.coefficients.tolist(), constraints))
    function_row = zeros(n).tolist()
    for i in range(n - 1):
        if i not in basis:
            function_row[i + 1] += function.coefficients[i]
        else:
            for constraint in constraints:
                if constraint.coefficients[i] == 0:
                    continue
                coefficients = constraint.coefficients
                for j in range(n - 1):
                    if j == i:
                        continue
                    function_row[j + 1] -= coefficients[j] * function.coefficients[i] / coefficients[i]
                function_row[0] -= constraint.value * function.coefficients[i] / coefficients[i]
    table.append(function_row)

    return table

def find_better_basis(table: [[float]], basis: [float], n: int, m: int):
    while True:
        max_simplex_diff = 0
        lead_element_j = -1
        for j in range(1, n):
            if table[m][j] > max_simplex_diff:
                max_simplex_diff = table[m][j]
                lead_element_j = j
        if max_simplex_diff < eps:
            break

        min_fraction = inf
        if table[0][lead_element_j] != 0:
            if sign(table[0][0]) == sign(table[0][lead_element_j]):
                min_fraction = table[0][0] / table[0][lead_element_j]
        lead_element_i = 0
        for i in range(m):
            fraction = inf
            if table[i][lead_element_j] != 0:
                fraction = table[i][0] / table[i][lead_element_j]

            if sign(table[i][0]) == sign(table[i][lead_element_j]) and fraction < min_fraction:
                min_fraction = fraction
                lead_element_i = i

        lead_element = table[lead_element_i][lead_element_j]
        for j in range(n):
            table[lead_element_i][j] /= lead_element

        for i in range(m + 1):
            if i == lead_element_i:
                continue

            d = -table[i][lead_element_j]
            for j in range(n):
                table[i][j] += d * table[lead_element_i][j]

        basis[lead_element_i] = lead_element_j - 1

def solve_lpt(lpt: LinearProgrammingTask, basic_feasible_solution: [float] = None) -> Constraint:
    optimize_max = lpt.function.optimize_max
    lpt.to_canonic_form()

    if not basic_feasible_solution:
        basic_feasible_solution = find_basic_feasible_solution(lpt)

    answer = simplex_method(lpt, basic_feasible_solution)
    answer.value *= (1 if optimize_max else -1)

    return answer

def find_basic_feasible_solution(lpt: LinearProgrammingTask) -> [float]:
    n, m = lpt.dimensions()
    basic_solution = np.zeros(n).tolist()
    function = Function(basic_solution + [-1] * m)
    constraints = []
    for i, constraint in enumerate(lpt.constraints):
        coefficients = []
        coefficients.extend(constraint.coefficients.tolist())
        coefficients.extend(zeros(i).tolist())
        coefficients.append(1)
        coefficients.extend(zeros(m - i - 1).tolist())

        value = constraint.value
        constraints.append(Constraint(coefficients, value, 0))
        if value == 0:
            value = inf

        basic_solution.append(value)

    solution_coefficients = solve_lpt(LinearProgrammingTask(function, constraints), basic_solution).coefficients
    return solution_coefficients[:n]

def simplex_method(lpt: LinearProgrammingTask, basic_feasible_solution: [float]) -> Constraint:
    n, m = lpt.dimensions()
    n += 1
    table, basis = build_table(lpt, basic_feasible_solution, n, m)
    find_better_basis(table, basis, n, len(lpt.constraints))

    coefficients = zeros(len(basic_feasible_solution)).tolist()
    n, m = lpt.dimensions()
    for i in range(m):
        coefficients[basis[i]] = table[i][0]

    value = -table[m][0]

    return Constraint(coefficients, value)

In [5]:
class TestExample:
    lpt: LinearProgrammingTask
    expected_answer: float
    basic_feasible_solution: [float] = None

    def __init__(self, lpt: LinearProgrammingTask, expected_answer: float, basic_feasible_solution: [float] = None):
        self.lpt = lpt
        self.expected_answer = expected_answer
        self.basic_feasible_solution = basic_feasible_solution

def test(test_example: TestExample) -> float:
    print("Test started")
    print("Task:")
    print(str(test_example.lpt))
    actual_answer = solve_lpt(test_example.lpt, test_example.basic_feasible_solution)
    if abs(actual_answer.value - test_example.expected_answer) < eps:
        print(f"Test succeeded, answer: {actual_answer.value}, point: {actual_answer.coefficients}")
        return actual_answer.value
    else:
        print(f"Test failed, expected: {test_example.expected_answer}, actual: {actual_answer.value}")
        return None

In [103]:
test_examples_with_basic_solution: [TestExample] = []

test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-6, -1, -4, 5], False),
        constraints=[
            Constraint([3, 1, -1, 1], 4),
            Constraint([5, 1, 1, -1], 4),
        ]
    ),
    expected_answer=-4.0,
    basic_feasible_solution=[1, 0, 0, 1]
))
test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-6, -1, -4, 5], False),
        constraints=[
            Constraint([3, 1, -1, 1], 4),
            Constraint([5, 1, 1, -1], 4),
        ]
    ),
    expected_answer=-4.0,
    basic_feasible_solution=None
))

test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -2, -3, 1], False),
        constraints=[
            Constraint([1, -3, -1, -2], -4),
            Constraint([1, -1, 1, 0], 0),
        ]
    ),
    expected_answer=-6.0,
    basic_feasible_solution=[0, 1, 1, 0]
))
test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -2, -3, 1], False),
        constraints=[
            Constraint([1, -3, -1, -2], -4),
            Constraint([1, -1, 1, 0], 0),
        ]
    ),
    expected_answer=-6.0,
    basic_feasible_solution=None
))

test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -2, -1, 3, -1], False),
        constraints=[
            Constraint([1, 1, 0, 2, 1], 5),
            Constraint([1, 1, 1, 3, 2], 9),
            Constraint([0, 1, 1, 2, 1], 6),
        ]
    ),
    expected_answer=-11.0,
    basic_feasible_solution=[0, 0, 1, 2, 1]
))
test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -2, -1, 3, -1], False),
        constraints=[
            Constraint([1, 1, 0, 2, 1], 5),
            Constraint([1, 1, 1, 3, 2], 9),
            Constraint([0, 1, 1, 2, 1], 6),
        ]
    ),
    expected_answer=-11.0,
    basic_feasible_solution=None
))

test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -1, -1, 1, -1], False),
        constraints=[
            Constraint([1, 1, 2, 0, 0], 4),
            Constraint([0, -2, -2, 1, -1], -6),
            Constraint([1, -1, 6, 1, 1], 12),
        ]
    ),
    expected_answer=-10.0,
    basic_feasible_solution=[1, 1, 2, 0, 0]
))
test_examples_with_basic_solution.append(TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -1, -1, 1, -1], False),
        constraints=[
            Constraint([1, 1, 2, 0, 0], 4),
            Constraint([0, -2, -2, 1, -1], -6),
            Constraint([1, -1, 6, 1, 1], 12),
        ]
    ),
    expected_answer=-10.0,
    basic_feasible_solution=None
))

In [104]:
for i in range(0, len(test_examples_with_basic_solution), 2):
    test_with_solution = test_examples_with_basic_solution[i]
    test_without_solution = test_examples_with_basic_solution[i + 1]
    ans1 = test(test_with_solution)
    print()
    ans2 = test(test_without_solution)
    if ans1 == ans2:
        print("Values are equal")
    else:
        print(f"Fail, test with solution answered {ans1}, without: {ans2}")

    print()
    print("=================================================================================")

Test started
Task:
-6.0x1 - x2 - 4.0x3 + 5.0x4 -> min
3.0x1 + x2 - x3 + x4 = 4
5.0x1 + x2 + x3 - x4 = 4
Test succeeded, answer: -4.0, point: [0. 4. 0. 0.]

Test started
Task:
-6.0x1 - x2 - 4.0x3 + 5.0x4 -> min
3.0x1 + x2 - x3 + x4 = 4
5.0x1 + x2 + x3 - x4 = 4
Test succeeded, answer: -4.0, point: [0. 4. 0. 0.]
Values are equal

Test started
Task:
-x1 - 2.0x2 - 3.0x3 + x4 -> min
x1 - 3.0x2 - x3 - 2.0x4 = -4
x1 - x2 + x3 = 0
Test succeeded, answer: -5.999999999999999, point: [2. 2. 0. 0.]

Test started
Task:
-x1 - 2.0x2 - 3.0x3 + x4 -> min
x1 - 3.0x2 - x3 - 2.0x4 = -4
x1 - x2 + x3 = 0
Test succeeded, answer: -5.999999999999999, point: [2. 2. 0. 0.]
Values are equal

Test started
Task:
-x1 - 2.0x2 - x3 + 3.0x4 - x5 -> min
x1 + x2 + 2.0x4 + x5 = 5
x1 + x2 + x3 + 3.0x4 + 2.0x5 = 9
x2 + x3 + 2.0x4 + x5 = 6
Test succeeded, answer: -11.0, point: [3. 2. 4. 0. 0.]

Test started
Task:
-x1 - 2.0x2 - x3 + 3.0x4 - x5 -> min
x1 + x2 + 2.0x4 + x5 = 5
x1 + x2 + x3 + 3.0x4 + 2.0x5 = 9
x2 + x3 + 2.0x4 + x

In [105]:
test_examples_without_solution: [TestExample] = [TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, 4, -3, 10], False),
        constraints=[
            Constraint([1, 1, -1, -10], 0),
            Constraint([1, 14, 10, -10], 11),
        ]
    ),
    expected_answer=-4.0,
    basic_feasible_solution=None
), TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, 5, 1, -1], False),
        constraints=[
            Constraint([1, 3, 3, 1], 3, -1),
            Constraint([2, 0, 3, -1], 4, -1),
        ]
    ),
    expected_answer=-3.0,
    basic_feasible_solution=None
), TestExample(
    lpt=LinearProgrammingTask(
        function=Function([-1, -1, 1, -1, 2], False),
        constraints=[
            Constraint([3, 1, 1, 1, -2], 10),
            Constraint([6, 1, 2, 3, -4], 20),
            Constraint([10, 1, 3, 6, -7], 30),
        ]
    ),
    expected_answer=10.0,
    basic_feasible_solution=None
)]

In [106]:
for test_example in test_examples_without_solution:
    test(test_example)
    print()
    print("=================================================================================")

Test started
Task:
-x1 + 4.0x2 - 3.0x3 + 10.0x4 -> min
x1 + x2 - x3 - 10.0x4 = 0
x1 + 14.0x2 + 10.0x3 - 10.0x4 = 11
Test succeeded, answer: -4.000000000000002, point: [1. 0. 1. 0.]

Test started
Task:
-x1 + 5.0x2 + x3 - x4 -> min
x1 + 3.0x2 + 3.0x3 + x4 <= 3
2.0x1 + 3.0x3 - x4 <= 4
Test succeeded, answer: -3.0, point: [2.33333333 0.         0.         0.66666667]

Test started
Task:
-x1 - x2 + x3 - x4 + 2.0x5 -> min
3.0x1 + x2 + x3 + x4 - 2.0x5 = 10
6.0x1 + x2 + 2.0x3 + 3.0x4 - 4.0x5 = 20
10.0x1 + x2 + 3.0x3 + 6.0x4 - 7.0x5 = 30
Test succeeded, answer: 9.99999999999999, point: [ 2.66453526e-15  0.00000000e+00  1.00000000e+01 -0.00000000e+00
  0.00000000e+00]



# Вопросы на защиту

### 1. Общая и каноническая форма задачи линейного программирования

#### Общая форма (для случая максимума):

$\begin{cases}
\sum_{j=1}^{n} c_{j}x_{j} \rightarrow max \\
\sum_{j=1}^{n} a_{ij}x_{j} \le b_{i}, i = \overline{1, m_1} \\
\sum_{j=1}^{n} a_{ij}x_{j} \ge b_{i}, i = \overline{m_{1}+1, m2} \\
\sum_{j=1}^{n} a_{ij}x_{j} = b_{i}, i = \overline{m_{2}+1, m} \\
\end{cases}$

#### Каноническая форма

$\begin{cases}
\sum_{j=1}^{n} c_{j}x_{j} \rightarrow max \\
\sum_{j=1}^{n} a_{ij}x_{j} = b_{i}, i = \overline{1, m} \\
x_{i} \ge 0, i = \overline{1, n}
\end{cases}$

### 2. Методы естественного базиса. Метод искусственного базиса.

#### Метод естественного базиса

1. Приводим ЗЛП в каноническую форму
2. Если матрица системы уравнений содержит единичную подматрицу m $\times$ m, то есть очевидное неотрицательное базисное решение
3. Проводим проверку решения на оптимальность, в случае неоптимальности переходим к другому решению с помощью преобразований Жордана-Гаусса:

    3.1 Вводим в базис переменную, давшую минимальную отрицательную величину симплекс-разности $\sum_{i}c_{i}a_{ij}$ - $c_{j}$

    3.2 Выводим из базиса переменную, давшую минимальное положительное отношение $\frac{b_i}{a_{ij}}, a_{ij} > 0$

    3.3 Пересчитываем элементы симплекс-таблицы

#### Метод искусственного базиса

Пусть дана задача в канонической форме в матричном виде
$\begin{cases}
\sum_{j=1}^{n} c_{j}x_{j} \rightarrow max \\
Ax = b \\
x \ge 0
\end{cases}$

и начальное базисное решение найти затруднительно. Тогда решим вспомогательную задачу

$\begin{cases}
g = -\sum_{i=1}^{m} y_{i} \rightarrow max \\
Ax + y = b \\
x \ge 0 \\
y \ge 0
\end{cases}$


, где вектор y - вектор искусственных переменных. Базисное решение этой задачи очевидно - x = 0, y = b. И тогда max(g) = max(0, $\ldots$, 0) = 0.
Следовательно, оптимальное решение вспомогательной задачи является допустимым решением основной задачи.

### 3. Доказать, что ОДР (область допустимых решений) является выпуклым множеством.

Пересечение выпуклых множеств является выпуклым множеством

### 4. Может ли ОДР в задаче линейного программирования состоять из одной единственной точки? Если да, то привести пример.

Да, может. Пример:

$\begin{cases}
-x \rightarrow max \\
x = 1
\end{cases}$

### 6. Найти все базиси системы равенств и соответствующие им базисные решения:

* x1: (1, 0, 0, 0)

* x3: (0, 0, 1, 0)

* x1, x3: (i, 0, j, 0), $\forall$ i, j: i + j = 1

### 7. В данной системе ограничений выразить базисные переменные указанного базисного допустимого решения x = $(1, 2, 0)^T$ через небазисные

Базисное допустимое решение [1, 2, 0]. Тогда базисные переменные - $x_1$ и $x_2$
Выразим их через $x_3$:

$x_1 = 1-x_3$

$x_2 = 2 - x_3$

### 8. Исследовать на оптимальность решение x = $(0, 0, 1, 1)^T$ задачи

In [6]:
task = LinearProgrammingTask(
    Function(
        [1, 1, -2, -3],
        False
    ),
    [
        Constraint(
            [2, -1, 1, 0,],
            1
        ),
        Constraint(
            [-1, 2, 0, 1],
            1
        )
    ]
)
answer = solve_lpt(
    task,
    [0, 0, 1, 1]
)
print(answer.value)
print(answer.coefficients)

-5.0
[0. 0. 1. 1.]


Минимум функции -5 находится в точке [0, 0, 1, 1] => предложенное решение оптимально